In [171]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GridSearchCV  
from sklearn.preprocessing import Imputer
from sklearn import linear_model
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.model_selection import train_test_split

In [172]:
rf = RandomForestClassifier(n_estimators=100000, n_jobs=-1, max_depth = 10)
lr = LogisticRegression(random_state=10, solver='liblinear', max_iter=1000000, penalty='l1', C=0.01, n_jobs=-1)


In [173]:
train_set = pd.read_csv("insurance_claims.csv")

In [174]:
train_set.shape

(1000, 40)

In [175]:
train_set.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


In [176]:
train_op_col=train_set['fraud_reported'][:]

In [177]:
train_set=train_set.drop('fraud_reported', axis=1)
train_set=train_set.drop('policy_bind_date', axis=1)
train_set=train_set.drop('incident_location', axis=1)
train_set=train_set.drop('policy_number', axis=1)
train_set['policy_csl']=train_set['policy_csl'].map({'250/500': 0.5, '100/300': 0.333333333333, '500/1000':  0.5})
train_set['insured_education_level']=train_set['insured_education_level'].map({'MD' : 6, 'PhD': 7, 'Associate': 4, 'Masters': 5, 'High School': 1, 'College': 2, 'JD': 3})
train_set['incident_severity']=train_set['incident_severity'].map({'Major Damage': 3, 'Minor Damage': 2, 'Total Loss': 4, 'Trivial Damage': 1})

In [178]:
train_set.head()

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,_c39
0,328,48,OH,0.500000,1000,1406.91,0,466132,MALE,6,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,NaN
1,228,42,IN,0.500000,2000,1197.22,5000000,468176,MALE,6,...,0,?,5070,780,780,3510,Mercedes,E400,2007,NaN
2,134,29,OH,0.333333,2000,1413.14,5000000,430632,FEMALE,7,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,NaN
3,256,41,IL,0.500000,2000,1415.74,6000000,608117,FEMALE,7,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,NaN
4,228,44,IL,0.500000,1000,1583.91,6000000,610706,MALE,4,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,NaN


In [179]:
lis=[]
str_lis=[]
for x in train_set:
    
    if(type(train_set[x][0]) != str):
        print(x)
        lis.append(x)
    else:
        str_lis.append(x)


months_as_customer
age
policy_csl
policy_deductable
policy_annual_premium
umbrella_limit
insured_zip
insured_education_level
capital-gains
capital-loss
incident_severity
incident_hour_of_the_day
number_of_vehicles_involved
bodily_injuries
witnesses
total_claim_amount
injury_claim
property_claim
vehicle_claim
auto_year
_c39


In [180]:
for x in train_set:
    if(train_set[x].dtype == "object"):
        print(x)

policy_state
insured_sex
insured_occupation
insured_hobbies
insured_relationship
incident_date
incident_type
collision_type
authorities_contacted
incident_state
incident_city
property_damage
police_report_available
auto_make
auto_model


In [181]:
train_set['auto_model'].unique()

array(['92x', 'E400', 'RAM', 'Tahoe', 'RSX', '95', 'Pathfinder', 'A5',
       'Camry', 'F150', 'A3', 'Highlander', 'Neon', 'MDX', 'Maxima',
       'Legacy', 'TL', 'Impreza', 'Forrestor', 'Escape', 'Corolla',
       '3 Series', 'C300', 'Wrangler', 'M5', 'X5', 'Civic', 'Passat',
       'Silverado', 'CRV', '93', 'Accord', 'X6', 'Malibu', 'Fusion',
       'Jetta', 'ML350', 'Ultima', 'Grand Cherokee'], dtype=object)

In [182]:
print(len(lis))

print(len(str_lis))

21
15


In [183]:
train_set=train_set[lis][:]

In [184]:
train_set.head()

,months_as_customer,age,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_education_level,capital-gains,capital-loss,...,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
0,328,48,0.500000,1000,1406.91,0,466132,6,53300,0,...,5,1,1,2,71610,6510,13020,52080,2004,NaN
1,228,42,0.500000,2000,1197.22,5000000,468176,6,0,0,...,8,1,0,0,5070,780,780,3510,2007,NaN
2,134,29,0.333333,2000,1413.14,5000000,430632,7,35100,0,...,7,3,2,3,34650,7700,3850,23100,2007,NaN
3,256,41,0.500000,2000,1415.74,6000000,608117,7,48900,-62400,...,5,1,1,2,63400,6340,6340,50720,2014,NaN
4,228,44,0.500000,1000,1583.91,6000000,610706,4,66000,-46000,...,20,1,0,1,6500,1300,650,4550,2009,NaN


In [185]:
fill_miss = Imputer(missing_values='NaN',strategy='median',copy=True)

In [186]:
train_dummy_data = pd.get_dummies(train_set)
complete_train_dummy_data = fill_miss.fit_transform(train_dummy_data.values)

In [187]:
complete_train_data = pd.DataFrame(complete_train_dummy_data)

In [188]:
complete_train_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,328.0,48.0,0.500000,1000.0,1406.91,0.0,466132.0,6.0,53300.0,0.0,3.0,5.0,1.0,1.0,2.0,71610.0,6510.0,13020.0,52080.0,2004.0
1,228.0,42.0,0.500000,2000.0,1197.22,5000000.0,468176.0,6.0,0.0,0.0,2.0,8.0,1.0,0.0,0.0,5070.0,780.0,780.0,3510.0,2007.0
2,134.0,29.0,0.333333,2000.0,1413.14,5000000.0,430632.0,7.0,35100.0,0.0,2.0,7.0,3.0,2.0,3.0,34650.0,7700.0,3850.0,23100.0,2007.0
3,256.0,41.0,0.500000,2000.0,1415.74,6000000.0,608117.0,7.0,48900.0,-62400.0,3.0,5.0,1.0,1.0,2.0,63400.0,6340.0,6340.0,50720.0,2014.0
4,228.0,44.0,0.500000,1000.0,1583.91,6000000.0,610706.0,4.0,66000.0,-46000.0,2.0,20.0,1.0,0.0,1.0,6500.0,1300.0,650.0,4550.0,2009.0


In [189]:
for x in complete_train_data:
    print(np.isnan(complete_train_data[x]).sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [190]:
complete_train_data.shape

(1000, 20)

In [191]:
train_op_col=train_op_col.map({'Y': 1, 'N': 0})

In [192]:
sample_train_set, sample_test_set, train_set_op_col, test_set_op_col = train_test_split(complete_train_data, train_op_col, test_size=0.3)

In [193]:
sample_train_set.shape

(700, 20)

In [194]:
sample_test_set.shape

(300, 20)

In [195]:
#pca=decomposition.PCA(n_components=10)
#pca_train_data = pca.fit_transform(sample_train_set)
#pca_test_data = pca.fit_transform(sample_test_set)

In [196]:
#pca_train_data = pd.DataFrame(pca_train_data)


In [197]:
#pca_test_data = pd.DataFrame(pca_test_data)

In [198]:
#lr.fit(sample_train_set, train_set_op_col)
rf.fit(sample_train_set, train_set_op_col)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100000, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [199]:
print(rf.feature_importances_)


[ 0.06003435  0.05130321  0.00980059  0.01534065  0.07335088  0.02239108
  0.07087857  0.03164129  0.03368031  0.04297053  0.16475813  0.04818334
  0.01523     0.01554359  0.02199449  0.06583475  0.0699542   0.07130032
  0.07123503  0.0445747 ]


In [200]:
#res = lr.predict(sample_test_set)
res = rf.predict(sample_test_set)

In [201]:
overfitting_op = rf.predict(sample_train_set)
#overfitting_op = lr.predict(sample_train_set)
overfitting_ans = metrics.accuracy_score(overfitting_op, train_set_op_col)
print(overfitting_ans)

0.998571428571


In [202]:
ans = metrics.accuracy_score(test_set_op_col, res)

In [203]:
print(sum(train_set_op_col==0))

532


In [204]:
print(ans)

0.763333333333


In [205]:
result_set = pd.read_csv("result_test.csv")

In [206]:
result_set.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,194,34,573572,1991-06-16,IL,100/300,500,1429.96,0,463307,...,YES,57200,11440,5720,40040,Toyota,Camry,2005,N,NaN
1,128,35,437960,2001-04-03,IN,250/500,1000,1074.99,0,453620,...,?,7590,1380,690,5520,Accura,MDX,2012,N,NaN
2,150,37,649800,2014-03-16,OH,500/1000,1000,1007.00,0,466238,...,?,80080,12320,12320,55440,Chevrolet,Silverado,2013,N,NaN
3,104,30,544225,2010-08-03,OH,100/300,500,1052.85,0,607697,...,YES,4800,960,480,3360,BMW,3 Series,2006,N,NaN
4,163,37,390256,2009-11-25,IN,500/1000,1000,1200.33,4000000,477631,...,YES,3900,390,780,2730,Volkswagen,Jetta,2008,Y,NaN


In [207]:
result_set=result_set.drop('fraud_reported', axis=1)
result_set=result_set.drop('policy_bind_date', axis=1)
result_set=result_set.drop('incident_location', axis=1)
result_set=result_set.drop('policy_number', axis=1)
result_set['policy_csl']=result_set['policy_csl'].map({'250/500': 0.5, '100/300': 0.333333333333, '500/1000':  0.5})
result_set['insured_education_level']=result_set['insured_education_level'].map({'MD' : 6, 'PhD': 7, 'Associate': 4, 'Masters': 5, 'High School': 1, 'College': 2, 'JD': 3})
result_set['incident_severity']=result_set['incident_severity'].map({'Major Damage': 3, 'Minor Damage': 2, 'Total Loss': 4, 'Trivial Damage': 1})

In [208]:
len(lis)

21

In [209]:
result_set = result_set[lis][:]

In [210]:
result_set

,months_as_customer,age,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_education_level,capital-gains,capital-loss,...,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
0,194,34,0.333333,500,1429.96,0,463307,3,67800,0,...,11,3,1,2,57200,11440,5720,40040,2005,NaN
1,128,35,0.500000,1000,1074.99,0,453620,4,0,-48800,...,16,1,2,0,7590,1380,690,5520,2012,NaN
2,150,37,0.500000,1000,1007.00,0,466238,7,30400,-89400,...,9,1,0,2,80080,12320,12320,55440,2013,NaN
3,104,30,0.333333,500,1052.85,0,607697,6,0,-70100,...,8,1,0,0,4800,960,480,3360,2006,NaN
4,163,37,0.500000,1000,1200.33,4000000,477631,1,0,-36400,...,7,1,2,1,3900,390,780,2730,2008,NaN
5,80,26,0.333333,1000,1343.00,0,443625,5,64600,0,...,12,1,0,0,90400,9040,9040,72320,1995,NaN
6,65,29,0.500000,2000,1441.60,5000000,472223,6,0,0,...,21,3,0,0,62900,6290,12580,44030,1998,NaN
7,179,32,0.333333,1000,1433.42,6000000,608328,4,53800,0,...,4,1,1,2,54200,5420,10840,37940,2014,NaN
8,372,50,0.500000,2000,1368.57,0,474860,6,0,0,...,18,3,2,2,51800,5180,10360,36260,2003,NaN
9,398,55,0.500000,500,862.19,0,606858,1,69400,0,...,7,1,2,3,6600,600,1200,4800,2012,NaN


In [211]:
fill_miss = Imputer(missing_values='NaN',strategy='median',copy=True)
result_dummy_data = pd.get_dummies(result_set)
complete_result_dummy_data = fill_miss.fit_transform(result_dummy_data.values)
complete_result_data = pd.DataFrame(complete_result_dummy_data)

In [212]:
result_dummy_data.head()

,months_as_customer,age,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_education_level,capital-gains,capital-loss,...,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
0,194,34,0.333333,500,1429.96,0,463307,3,67800,0,...,11,3,1,2,57200,11440,5720,40040,2005,NaN
1,128,35,0.500000,1000,1074.99,0,453620,4,0,-48800,...,16,1,2,0,7590,1380,690,5520,2012,NaN
2,150,37,0.500000,1000,1007.00,0,466238,7,30400,-89400,...,9,1,0,2,80080,12320,12320,55440,2013,NaN
3,104,30,0.333333,500,1052.85,0,607697,6,0,-70100,...,8,1,0,0,4800,960,480,3360,2006,NaN
4,163,37,0.500000,1000,1200.33,4000000,477631,1,0,-36400,...,7,1,2,1,3900,390,780,2730,2008,NaN


In [213]:
results = rf.predict(complete_result_data)
print(results)

[0 0 0 0 1 0 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0]
